In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('../data/currency_data.csv')
df.head()

,Date,USD_INR,EUR_INR,JPY_INR,CNY_INR,GBP_INR
0,NaN,USDINR=X,EURINR=X,JPYINR=X,CNYINR=X,GBPINR=X
1,2024-10-15,84.04850006103516,91.60199737548828,0.5609909892082214,NaN,109.80130004882812
2,2024-10-16,84.05619812011719,91.41999816894531,0.5633090138435364,NaN,109.85874938964844
3,2024-10-17,84.03929901123047,91.13999938964844,0.5611079931259155,NaN,109.15906524658203
4,2024-10-18,84.05380249023438,90.95600128173828,0.5595570206642151,NaN,109.41233825683594


In [4]:
df = df.drop(columns=['CNY_INR'])

In [5]:
# Drop the first row if it's headers repeated as data
df = df.iloc[1:]

In [6]:
df.head()

,Date,USD_INR,EUR_INR,JPY_INR,GBP_INR
1,2024-10-15,84.04850006103516,91.60199737548828,0.5609909892082214,109.80130004882812
2,2024-10-16,84.05619812011719,91.41999816894531,0.5633090138435364,109.85874938964844
3,2024-10-17,84.03929901123047,91.13999938964844,0.5611079931259155,109.15906524658203
4,2024-10-18,84.05380249023438,90.95600128173828,0.5595570206642151,109.41233825683594
5,2024-10-21,84.0687026977539,90.16999816894531,0.5549439787864685,109.71588134765625


In [7]:
df.columns

Index(['Date', 'USD_INR', 'EUR_INR', 'JPY_INR', 'GBP_INR'], dtype='object')

In [8]:
df.shape

(259, 5)

In [9]:
df.dtypes

Date       object
USD_INR    object
EUR_INR    object
JPY_INR    object
GBP_INR    object
dtype: object

In [10]:
df = df.astype(
    {
        "Date": str,
        "USD_INR": float,
        "EUR_INR": float,
        "JPY_INR": float,
        "GBP_INR": float,
    }
)

In [13]:
df.dtypes

Date        object
USD_INR    float64
EUR_INR    float64
JPY_INR    float64
GBP_INR    float64
dtype: object

In [18]:
cols = [col for col in df.columns if col != 'Date']
fig = px.line(
    df,
    x="Date",
    y=cols,
    title=f"Values of currencies wrt INR",
)
fig.show()

In [19]:
df.drop(columns=["Date"]).pct_change()

,USD_INR,EUR_INR,JPY_INR,GBP_INR
1,NaN,NaN,NaN,NaN
2,0.000092,-0.001987,0.004132,0.000523
3,-0.000201,-0.003063,-0.003907,-0.006369
4,0.000173,-0.002019,-0.002764,0.002320
5,0.000177,-0.008642,-0.008244,0.002774
...,...,...,...,...
255,0.000390,-0.001664,-0.003654,-0.001091
256,0.000994,-0.004575,-0.000943,-0.006310
257,-0.001216,-0.000487,0.003105,0.002382
258,-0.001039,-0.001627,-0.001539,-0.002505


In [20]:
pct_changes = df.drop(columns=["Date"]).pct_change()
avg_change = pct_changes.mean() * 100  # Convert to percentage
avg_change

USD_INR    0.017821
EUR_INR    0.045435
JPY_INR    0.017536
GBP_INR    0.027744
dtype: float64

In [21]:
# Compute daily percentage change
pct_change = df.copy()
for col in df.columns:
    if col != "Date":
        pct_change[col] = df[col].pct_change() * 100 

In [22]:
cum_change = pct_change.copy()
for col in df.columns:
    if col != "Date":
        cum_change[col] = pct_change[col].cumsum()

In [23]:
norm = df.copy()
for col in df.columns:
    if col != "Date":
        norm[col] = (df[col] / df[col].iloc[0]) * 100

In [25]:
import plotly.express as px

fig = px.line(
    norm,
    x="Date",
    y=norm.columns[1:],  # all except Date
    title="Rupee Performance vs Major Currencies (Base = 100)",
    labels={"value": "Index (Base=100)", "Date": "Date"},
)

fig.update_layout(
    yaxis_title="Rupee Value Index", legend_title="Currency", hovermode="x unified"
)

fig.add_hline(y=100, line_dash="dot", line_color="gray", annotation_text="Base Level")

fig.show()